In [1]:
var('t q')

def L_partition_vector2(partition, vector):
    for i in range(len(partition)):
        vector = d[-partition[len(partition) - i - 1]]*vector
    return vector

def my_coefficient(partition, vector):
    return vector.monomial_coefficients().get(tuple(-k for k in partition), 0)

def series_t_q(list_of_list_of_partition):
    series = 0
    for i in range(len(list_of_list_of_partition)):
        series_t = 0
        for j in range(len(list_of_list_of_partition[i])):
            series_t = series_t + t^len(list_of_list_of_partition[i][j])
        series = series + series_t*q^i
    return series

def partition_ends(partition, tail, is_greater):
    if len(partition) < len(tail) - 1:
        return False
    partition2 = partition[len(partition) - len(tail) + 1: len(partition)]
    if partition2 != tail[1:]:
        return False
    if is_greater:
        if partition == partition2:
            return True
        if partition[len(partition) - len(tail)] > tail[0]:
            return True
    else:
        if partition == partition2:
            return False
        if partition[len(partition) - len(tail)] == tail[0]:
            return True
    return False

def list_of_list_of_partition_ends(list_of_list_of_partition, tail, is_greater):
    return [[partition
             for partition in list_of_partition if partition_ends(partition, tail, is_greater)]
            for list_of_partition in list_of_list_of_partition]

def my_order(partition):
    polynomial = Partition(partition).to_exp(sum(partition))
    polynomial = [-k for k in polynomial]
    polynomial.reverse()
    return ((len(partition), polynomial)) 

def ordered_partitions(N):
    if N <= 0:
        return [[]]
    list_of_partition = list(Partitions(N))
    list_of_partition.sort(key = my_order, reverse = True)
    return list_of_partition

def find_singular_vector_at_weight(weight):
    list_of_partition = ordered_partitions(weight)
    list_of_partition.reverse()
    list_of_partition2 = ordered_partitions(weight - 1)
    list_of_partition2.reverse() 
    list_of_partition3 = ordered_partitions(weight - 2)
    list_of_partition3.reverse()
    matrix_1 = matrix({(i, j): my_coefficient(list_of_partition2[i], d[1]*L_partition_vector2(list_of_partition[j], v)) for j in range(len(list_of_partition)) for i in range(len(list_of_partition2))}, sparse = False)
    matrix_2 = matrix({(i, j): my_coefficient(list_of_partition3[i], d[2]*L_partition_vector2(list_of_partition[j], v)) for j in range(len(list_of_partition)) for i in range(len(list_of_partition3))}, sparse = False)
    matrix_1 = matrix_1.stack(matrix_2)
    matrix_1 = matrix_1.rref()
    vector = [-matrix_1[k, len(list_of_partition)-1] for k in range(len(list_of_partition)-1)]
    vector_list = list(reversed(vector + [1]))
    denominators = [value.denominator() for value in vector_list]
    common_denominator = LCM(denominators)
    vector_list = [value*common_denominator for value in vector_list]
    list_of_partition.reverse()
    vector2 = {tuple(list_of_partition[i]): vector_list[i] for i in range(len(list_of_partition))}
    return vector2

def find_singular_vectors():
    return (find_singular_vector_at_weight(m*n), find_singular_vector_at_weight((pp - m)*(qq - n)))

def vector_sum(vector1, vector2):
    return {partition: vector1.get(partition, 0) + vector2.get(partition, 0)
            for partition in set(vector1)|set(vector2)}

def vector_simplify0(partition):
    for i in range(len(partition) - 1):
        if partition[i] < partition[i + 1]:
            return {partition[0: i] + (partition[i] + partition[i + 1],)
                    + partition[i + 2: len(partition)]: partition[i + 1] - partition[i],
                    partition[0: i] + (partition[i + 1],) + (partition[i],)
                    + partition[i + 2: len(partition)]: 1}
    return {partition: 1}

def vector_simplify(vector):
    vector2 = {}
    for partition in set(vector):
        vector3 = vector_simplify0(partition)
        vector2 = vector_sum(vector2, {partition2: vector3[partition2]*vector[partition]
                                       for partition2 in set(vector3)})
    if vector2 == vector:
        return vector2
    return vector_simplify(vector2)

def L_partition_vector(partition, vector):
    return vector_simplify(
        {partition + partition2: vector[partition2] for partition2 in set(vector)})

def matrix_UVir_m1_m2(N):
    singular_vectors = find_singular_vectors()
    m1 = singular_vectors[0]
    m2 = singular_vectors[1]
    list_of_partition = ordered_partitions(N)
    list_of_partition2 = ordered_partitions(N - m*n)
    list_of_partition3 = ordered_partitions(N - (pp - m)*(qq - n))
    list_of_vector_m1 = [L_partition_vector(tuple(list_of_partition2[i]), m1)
                         for i in range(len(list_of_partition2))]
    list_of_vector_m2 = [L_partition_vector(tuple(list_of_partition3[i]), m2)
                         for i in range(len(list_of_partition3))]
    matrix_m1 = matrix({(i, j): list_of_vector_m1[i].get(tuple(list_of_partition[j]), 0)
                        for j in range(len(list_of_partition))
                        for i in range(len(list_of_partition2))}, sparse = False)
    matrix_m2 = matrix({(i, j): list_of_vector_m2[i].get(tuple(list_of_partition[j]), 0)
                        for j in range(len(list_of_partition))
                        for i in range(len(list_of_partition3))}, sparse = False)
    matrix_m1 = matrix_m1.stack(matrix_m2)
    matrix_m1 = matrix_m1.rref()
    return matrix_m1

def pivots_up_to(N):
    list_of_list_of_partition = []
    for i in range(N + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [list_of_partition[j] for j in matrix_UVir_m1_m2(i).pivots()])
    return list_of_list_of_partition

def basis_up_to(N):
    list_of_list_of_partition = []
    pivots = pivots_up_to(N)
    for i in range(N + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition if partition not in pivots[i]])
    return list_of_list_of_partition

def is_subpartition(partition, partition2):
    if len(partition) < len(partition2):
        return False
    for i in range(len(partition) - len(partition2) + 1):
        if partition[i: i + len(partition2)] == partition2:
            return True
    return False

def partition_contains_any_of_list(partition, list_of_partition):
    for partition2 in list_of_partition:
        if is_subpartition(partition, partition2):
            return True
    return False

def partition_contains_any_of_R(partition, s, list_of_partition):
    if partition == []:
        return False
    if partition_contains_any_of_list(partition, list_of_partition):
        return True
    if len(partition) < s:
        return False
    for i in range(len(partition) - s + 1):
        if partition[i] - partition[i + s - 1] < 2:
            return True
    return False

def basis_up_tov2(N, s, list_of_partition):
    list_of_list_of_partition = []
    for i in range(N + 1):
        list_of_partition2 = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition2
             if not partition_contains_any_of_R(partition, s, list_of_partition)])
    return list_of_list_of_partition

In [2]:
ordered_partitions(6)

[[1, 1, 1, 1, 1, 1],
 [2, 1, 1, 1, 1],
 [2, 2, 1, 1],
 [3, 1, 1, 1],
 [2, 2, 2],
 [3, 2, 1],
 [4, 1, 1],
 [3, 3],
 [4, 2],
 [5, 1],
 [6]]

In [3]:
Vir = lie_algebras.VirasoroAlgebra(QQ)
pp = 2
ss = 5
m = 1
n = 2
NN = 20
qq = 2*ss + 1
c = 1 - 6*(pp - qq)^2/(pp*qq)
h = ((n*pp - m*qq)^2 - (pp - qq)^2)/(4*pp*qq)
M = Vir.verma_module(c, h)
d = Vir.basis()
v = M.highest_weight_vector()

In [4]:
(c, h)

(-232/11, -4/11)

In [5]:
%time basis_up_to(NN) == basis_up_tov2(NN, ss, [[1, 1]])

CPU times: user 5min 42s, sys: 425 ms, total: 5min 42s
Wall time: 6min 11s


True

In [6]:
%time w = basis_up_tov2(25, ss, [[1, 1]])

CPU times: user 119 ms, sys: 9.24 ms, total: 129 ms
Wall time: 138 ms


In [7]:
series_t_q(w)

(9*t^8 + 59*t^7 + 126*t^6 + 147*t^5 + 109*t^4 + 51*t^3 + 12*t^2 + t)*q^25 + (5*t^8 + 41*t^7 + 101*t^6 + 123*t^5 + 97*t^4 + 47*t^3 + 12*t^2 + t)*q^24 + (2*t^8 + 28*t^7 + 78*t^6 + 103*t^5 + 84*t^4 + 43*t^3 + 11*t^2 + t)*q^23 + (t^8 + 18*t^7 + 60*t^6 + 85*t^5 + 74*t^4 + 39*t^3 + 11*t^2 + t)*q^22 + (11*t^7 + 45*t^6 + 70*t^5 + 63*t^4 + 36*t^3 + 10*t^2 + t)*q^21 + (6*t^7 + 33*t^6 + 56*t^5 + 55*t^4 + 32*t^3 + 10*t^2 + t)*q^20 + (3*t^7 + 23*t^6 + 45*t^5 + 46*t^4 + 29*t^3 + 9*t^2 + t)*q^19 + (t^7 + 16*t^6 + 35*t^5 + 39*t^4 + 26*t^3 + 9*t^2 + t)*q^18 + (10*t^6 + 27*t^5 + 32*t^4 + 23*t^3 + 8*t^2 + t)*q^17 + (6*t^6 + 20*t^5 + 27*t^4 + 20*t^3 + 8*t^2 + t)*q^16 + (3*t^6 + 15*t^5 + 21*t^4 + 18*t^3 + 7*t^2 + t)*q^15 + (t^6 + 10*t^5 + 17*t^4 + 15*t^3 + 7*t^2 + t)*q^14 + (7*t^5 + 13*t^4 + 13*t^3 + 6*t^2 + t)*q^13 + (4*t^5 + 10*t^4 + 11*t^3 + 6*t^2 + t)*q^12 + (2*t^5 + 7*t^4 + 9*t^3 + 5*t^2 + t)*q^11 + (t^5 + 5*t^4 + 7*t^3 + 5*t^2 + t)*q^10 + (3*t^4 + 6*t^3 + 4*t^2 + t)*q^9 + (2*t^4 + 4*t^3 + 4*t^2 + t)*

In [8]:
series_t_q(list_of_list_of_partition_ends(w, [1], True))

(t^8 + 18*t^7 + 58*t^6 + 83*t^5 + 72*t^4 + 40*t^3 + 11*t^2 + t)*q^25 + (t^8 + 12*t^7 + 46*t^6 + 69*t^5 + 64*t^4 + 37*t^3 + 11*t^2 + t)*q^24 + (7*t^7 + 33*t^6 + 56*t^5 + 54*t^4 + 33*t^3 + 10*t^2 + t)*q^23 + (4*t^7 + 25*t^6 + 46*t^5 + 47*t^4 + 30*t^3 + 10*t^2 + t)*q^22 + (2*t^7 + 17*t^6 + 36*t^5 + 39*t^4 + 27*t^3 + 9*t^2 + t)*q^21 + (t^7 + 12*t^6 + 29*t^5 + 34*t^4 + 24*t^3 + 9*t^2 + t)*q^20 + (7*t^6 + 22*t^5 + 27*t^4 + 21*t^3 + 8*t^2 + t)*q^19 + (5*t^6 + 17*t^5 + 23*t^4 + 19*t^3 + 8*t^2 + t)*q^18 + (2*t^6 + 12*t^5 + 18*t^4 + 16*t^3 + 7*t^2 + t)*q^17 + (t^6 + 9*t^5 + 15*t^4 + 14*t^3 + 7*t^2 + t)*q^16 + (6*t^5 + 11*t^4 + 12*t^3 + 6*t^2 + t)*q^15 + (4*t^5 + 9*t^4 + 10*t^3 + 6*t^2 + t)*q^14 + (2*t^5 + 6*t^4 + 8*t^3 + 5*t^2 + t)*q^13 + (t^5 + 5*t^4 + 7*t^3 + 5*t^2 + t)*q^12 + (3*t^4 + 5*t^3 + 4*t^2 + t)*q^11 + (2*t^4 + 4*t^3 + 4*t^2 + t)*q^10 + (t^4 + 3*t^3 + 3*t^2 + t)*q^9 + (t^4 + 2*t^3 + 3*t^2 + t)*q^8 + (t^3 + 2*t^2 + t)*q^7 + (t^3 + 2*t^2 + t)*q^6 + (t^2 + t)*q^5 + (t^2 + t)*q^4 + q^3*t 

In [9]:
series_t_q(list_of_list_of_partition_ends(w, [1, 1], True))

(8*t^8 + 41*t^7 + 68*t^6 + 64*t^5 + 37*t^4 + 11*t^3 + t^2)*q^25 + (4*t^8 + 29*t^7 + 55*t^6 + 54*t^5 + 33*t^4 + 10*t^3 + t^2)*q^24 + (2*t^8 + 21*t^7 + 45*t^6 + 47*t^5 + 30*t^4 + 10*t^3 + t^2)*q^23 + (t^8 + 14*t^7 + 35*t^6 + 39*t^5 + 27*t^4 + 9*t^3 + t^2)*q^22 + (9*t^7 + 28*t^6 + 34*t^5 + 24*t^4 + 9*t^3 + t^2)*q^21 + (5*t^7 + 21*t^6 + 27*t^5 + 21*t^4 + 8*t^3 + t^2)*q^20 + (3*t^7 + 16*t^6 + 23*t^5 + 19*t^4 + 8*t^3 + t^2)*q^19 + (t^7 + 11*t^6 + 18*t^5 + 16*t^4 + 7*t^3 + t^2)*q^18 + (8*t^6 + 15*t^5 + 14*t^4 + 7*t^3 + t^2)*q^17 + (5*t^6 + 11*t^5 + 12*t^4 + 6*t^3 + t^2)*q^16 + (3*t^6 + 9*t^5 + 10*t^4 + 6*t^3 + t^2)*q^15 + (t^6 + 6*t^5 + 8*t^4 + 5*t^3 + t^2)*q^14 + (5*t^5 + 7*t^4 + 5*t^3 + t^2)*q^13 + (3*t^5 + 5*t^4 + 4*t^3 + t^2)*q^12 + (2*t^5 + 4*t^4 + 4*t^3 + t^2)*q^11 + (t^5 + 3*t^4 + 3*t^3 + t^2)*q^10 + (2*t^4 + 3*t^3 + t^2)*q^9 + (t^4 + 2*t^3 + t^2)*q^8 + (t^4 + 2*t^3 + t^2)*q^7 + (t^3 + t^2)*q^6 + (t^3 + t^2)*q^5 + q^4*t^2 + q^3*t^2 + q*t